# Capstone Project Week 3 Assignment Part I

## 1. Transform Wikipedia page data into Dataframe

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [2]:
from IPython.display import display_html
import pandas as pd

unfiltered_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0] #read HTML from wikipedia
unfiltered_df.columns = ['PostalCode', 'Borough', 'Neighborhood'] #The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
unfiltered_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


As we see table above. There are NaN data on the Table. So we replace it to 'Not assigned'

In [3]:
unfiltered_df["Neighborhood"] = unfiltered_df["Neighborhood"].fillna('Not assigned') #remove NaN to 'Not Assigned'
unfiltered_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Drop rows which Borough equals Not assigned:

In [4]:
indexNames = unfiltered_df[ unfiltered_df['Borough'] == 'Not assigned' ].index
unfiltered_df.drop(indexNames , inplace=True)
unfiltered_df = unfiltered_df.reset_index(drop=True)
unfiltered_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


After that, group Neighborhood by PostalCode

In [5]:
unfiltered_df = unfiltered_df.groupby('PostalCode', as_index=False).agg(lambda x : ','.join(set(x)))
unfiltered_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally, filter the PostalCodes required by the assignment and order them accordingly:

In [6]:
filtered_df = unfiltered_df[unfiltered_df['PostalCode'].isin(['M5G', 'M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A']) ]
filtered_df = filtered_df.reset_index(drop=True)
filtered_df = filtered_df.loc[[8,3,4,1,5,6,2,11,10,9,0,7], :].reset_index(drop=True)
filtered_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


In [7]:
print('The dataframe shape is: ', filtered_df.shape)

The dataframe shape is:  (12, 3)


## 2. Add latitude and longitude from Geocoder

Use the Geocoder package or the csv file to create the dataframe.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [8]:
!wget -q -O 'geospatialdata.csv' https://cocl.us/Geospatial_data

In [9]:
df_geospatial = pd.read_csv('geospatialdata.csv') #read csv from geospatialdata
df_geospatial.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_geospatial.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
filtered_df = pd.merge(filtered_df, df_geospatial[['PostalCode', 'Latitude', 'Longitude']], on='PostalCode') #merge data
filtered_df[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']] #show data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420


In [12]:
print('The dataframe shape is: ', filtered_df.shape)

The dataframe shape is:  (12, 5)


## 3. Cluster the neighborhoods in Toronto

We are using Geocoder to find the Coordinates of Toronto

In [13]:
from geopy.geocoders import Nominatim 

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

lat_lng_coords = None

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map using our previous defined dataframe:

In [15]:
import folium 
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)
neighborhoods = filtered_df

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
       [lat, lng],
       radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
    parse_html=False).add_to(map_tor)  
    
map_tor

Define our Foursquare credentials:

In [16]:
CLIENT_ID = 'Y5JRMWFFBTTMXYD4TKCF5E2C5L52GR5AGTLUVE0KZX1OYGCU' # your Foursquare ID
CLIENT_SECRET = 'PCKWGHAZHPIT5RZET4FMWWCVTM0P4FVKQMPTPZY1QLGWFAT0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y5JRMWFFBTTMXYD4TKCF5E2C5L52GR5AGTLUVE0KZX1OYGCU
CLIENT_SECRET:PCKWGHAZHPIT5RZET4FMWWCVTM0P4FVKQMPTPZY1QLGWFAT0


Let's explore the first neighborhood in our dataset:

In [17]:
neighborhoods.loc[0, 'Neighborhood']
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


Get the top 100 venues from Central Bay Street:

In [18]:
LIMIT = 100
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

#url # display URL
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
results = requests.get(url).json()

#results
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Get a list of nearby venues:

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Somethin' 2 Talk About,Middle Eastern Restaurant,43.658395,-79.385338
3,Bubble Bath & Spa,Spa,43.659050,-79.385344
4,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

63 venues were returned by Foursquare.


Now lets do the same for all neighborhoods in our dataframe:

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'])

Central Bay Street
Hillcrest Village
Parkview Hill / Woodbine Gardens
Scarborough Village
Leaside
Studio District
Wexford / Maryvale
South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens
Humber Summit
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Malvern / Rouge
Regent Park / Harbourfront


In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(239, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Bay Street,43.657952,-79.387383,Jimmy's Coffee,43.658421,-79.385613,Coffee Shop
1,Central Bay Street,43.657952,-79.387383,Tim Hortons,43.658570,-79.385123,Coffee Shop
2,Central Bay Street,43.657952,-79.387383,Somethin' 2 Talk About,43.658395,-79.385338,Middle Eastern Restaurant
3,Central Bay Street,43.657952,-79.387383,Bubble Bath & Spa,43.659050,-79.385344,Spa
4,Central Bay Street,43.657952,-79.387383,Neo Coffee Bar,43.660140,-79.385870,Coffee Shop


In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Store,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Store,Wine Bar,Wine Shop
0,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.00,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
1,Central Bay Street,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000,0.015873,0.000000,0.015873,0.015873,0.000000,0.015873,0.000000,0.000,0.000000
2,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
3,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
4,Leaside,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000,0.000000,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
5,Malvern / Rouge,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
6,Parkview Hill / Woodbine Gardens,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
7,Regent Park / Harbourfront,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020833,...,0.000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000,0.020833
8,Scarborough Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
9,South Steeles / Silverstone / Humbergate / Jam...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000,0.000000


In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3          Boutique  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.21
1  Italian Restaurant  0.06
2      Sandwich Place  0.05
3                Café  0.05
4      Ice Cream Shop  0.03


----Hillcrest Village----
                      venue  freq
0                   Dog Run  0.25
1  Mediterranean Restaurant  0.25
2                      Pool  0.25
3               Golf Course  0.25
4           Harbor / Marina  0.00


----Humber Summit----
                   venue  freq
0            Pizza Place   0.5
1    Empanada Restaurant   0.5
2              Juice Bar   0.0
3  Performing Arts Venue   0.0
4                   Park   0.0


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1 

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Plane,Rental Car Location,Bar
1,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Ice Cream Shop,Fried Chicken Joint,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint
2,Hillcrest Village,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Wine Shop,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant
3,Humber Summit,Pizza Place,Empanada Restaurant,Wine Shop,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store
4,Leaside,Sporting Goods Shop,Coffee Shop,Bank,Furniture / Home Store,Burger Joint,Dessert Shop,Gym,Bike Shop,Beer Store,Breakfast Spot


In [32]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 3, 2, 1, 0, 1, 1, 4, 1], dtype=int32)

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Ice Cream Shop,Fried Chicken Joint,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint
1,M2H,North York,Hillcrest Village,43.803762,-79.363452,3,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Wine Shop,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937,1,Pizza Place,Gastropub,Gym / Fitness Center,Pet Store,Pharmacy,Bank,Athletics & Sports,Intersection,Fast Food Restaurant,Empanada Restaurant
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Playground,Convenience Store,Jewelry Store,Wine Shop,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop
4,M4G,East York,Leaside,43.709060,-79.363452,1,Sporting Goods Shop,Coffee Shop,Bank,Furniture / Home Store,Burger Joint,Dessert Shop,Gym,Bike Shop,Beer Store,Breakfast Spot


In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,0,Fast Food Restaurant,Wine Shop,Discount Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Ice Cream Shop,Fried Chicken Joint,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint
2,East York,1,Pizza Place,Gastropub,Gym / Fitness Center,Pet Store,Pharmacy,Bank,Athletics & Sports,Intersection,Fast Food Restaurant,Empanada Restaurant
4,East York,1,Sporting Goods Shop,Coffee Shop,Bank,Furniture / Home Store,Burger Joint,Dessert Shop,Gym,Bike Shop,Beer Store,Breakfast Spot
5,East Toronto,1,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Brewery,Diner,Middle Eastern Restaurant,Latin American Restaurant,Italian Restaurant
6,Scarborough,1,Auto Garage,Middle Eastern Restaurant,Breakfast Spot,Shopping Mall,Bakery,Sandwich Place,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant
7,Etobicoke,1,Grocery Store,Fried Chicken Joint,Coffee Shop,Beer Store,Liquor Store,Pharmacy,Pizza Place,Fast Food Restaurant,Video Store,Sandwich Place
9,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Plane,Rental Car Location,Bar
11,Downtown Toronto,1,Coffee Shop,Park,Bakery,Pub,Café,Restaurant,Theater,Breakfast Spot,French Restaurant,Gym / Fitness Center


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,North York,2,Pizza Place,Empanada Restaurant,Wine Shop,Diner,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store
